<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## Tracking Invitation Homes Activity Over Time

In this notebook, at Invitation Homes activity in five tertiary markets to answer the following questions:
- How many homes does Invitation Homes own in these markets today? How many did they own a year ago?
- What is the rent rate that Invitation Homes is charging renters in these homes? What was it a year ago? 
- How have these metrics changed year over year? 

The notebook is broken up into the following sections:
1. [Import required packages and setup the Parcl Labs API key](#1-import-required-packages-and-setup-the-parcl-labs-api-key)
2. [Pull Parcl IDs for Five Tertiary Markets](#2-Pull-parcl-iDs-for-five-tertiary-markets) 
3. [Retrieve the Data for the Current Invitation Homes Properties](#3-Retrieve-the-Data-for-the-Current-Invitation-Homes-Properties)
4. [Prepare the Data for the Current Invitation Homes Properties](#4-Prepare-the-Data-for-the-Current-Invitation-Homes-Properties)
5. [Retrieve the Data for the Nov 2023 Invitation Homes Properties](#5-Retrieve-the-Data-for-the-Nov-2023-Invitation-Homes-Properties)
6. [Prepare the Data for the November 2023 Invitation Homes Properties](#6-Prepare-the-Data-for-the-November-2023-Invitation-Homes-Properties)
7. [Calculate the YoY Change for the Invitation Homes Portfolio in the Tertiary Markets](#7-Calculate-the-YoY-Change-for-the-Invitation-Homes-Portfolio-in-the-Tertiary-Markets)

#### What will you create in this notebook?

##### Understand changes in supply and Demand YoY
<p align="center">
  <img src="../../../images/INVH_YoY_Changes_tertiary_markets.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/investor_analytics/invh_yoy_change.ipynb)

### 1. Import required packages and setup the Parcl Labs API key

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [3]:
import os
import pandas as pd
from parcllabs import ParclLabsClient
from requests.exceptions import RequestException

In [4]:
api_key = os.getenv('PARCL_LABS_API_KEY')
client = ParclLabsClient(api_key, turbo_mode=True)

### 2. Pull Parcl IDs for Five Tertiary Markets

In [5]:
# List of markets
queries = ['St. Louis', 'Indianapolis', 'Cincinnati', 'Columbia, SC', 'Columbus']

# Empty list to store parcl_ids
market_parcl_id_list = []

# Loop through each query and make the API request
for query in queries:
    markets = client.search.markets.retrieve(
        query=query,
        location_type='CBSA'
    )

    # Append Parcl IDs
    parcl_id = markets['parcl_id'].iloc[0] if hasattr(markets, 'iloc') else markets['parcl_id']
    market_parcl_id_list.append(parcl_id)

market_parcl_id_list

[2900321, 2899979, 2899647, 2899666, 2899671]

### 3. Retrieve the Data for the Current Invitation Homes Properties

In [6]:
# pass in the parcl IDs and use the invitation homes parameter to pull current IH properties in the 5 markets
current_ih_units = client.property.search.retrieve(
    parcl_ids=market_parcl_id_list,
    property_type='single_family',
    current_entity_owner_name='invitation_homes'
)

#Dataframe of properties with CBSA and Current Owner 
current_ih_units_df = current_ih_units[['parcl_property_id', 'cbsa', 'cbsa_parcl_id', 'current_entity_owner_name']]

#Create Parcl Prop ID list to pass into events
parcl_property_id_list = current_ih_units_df['parcl_property_id'].tolist()

parcl_property_id_list

Processing Parcl IDs |████████████████████████████████████████| 5/5 [100%] in 41.5s (0.12/s) 


[137699379,
 122336226,
 127064185,
 172792635,
 85391204,
 150074832,
 123635786,
 170396679,
 161303402,
 114279817,
 72483336,
 127169943,
 115040612,
 95837594,
 95216776,
 113994902,
 98400902,
 128513649,
 92271753,
 124918558,
 121667615,
 144876390,
 164603389,
 99967671,
 77161050,
 156028422,
 142044067,
 159697306,
 139928345,
 181105053,
 116171830,
 88673365,
 159340412,
 169408984,
 165778895,
 82850933,
 99605381,
 109944280,
 155332887,
 77965418,
 84234225,
 182833375,
 77720122,
 161157585,
 76704399,
 142467920,
 152305594,
 120564562,
 134513193,
 125645635,
 109420635,
 167707209,
 100437168,
 169382118,
 159786040,
 123792614,
 121044008,
 148982335,
 129467782,
 153069730,
 96849269,
 73650368,
 112512052,
 73996984,
 80354474,
 163556109,
 89091124,
 119271622,
 173910875,
 75526119,
 136321582,
 74258825,
 158409019,
 160381634,
 176940825,
 142531637,
 74039497,
 148728138,
 171793028,
 139993069,
 98881704,
 134533233,
 129391675,
 85715955,
 86691140,
 14153

In [7]:
# pass in the parcl_property_id_list and use the RENTAL parameter to pull in all rental events at the properties
rental_events = client.property.events.retrieve(
        parcl_property_ids=parcl_property_id_list,
        event_type='RENTAL',
        end_date='2024-11-30'
)

rental_events

Processing Parcl Property IDs |████████████████████████████████████████| 232/232 [100%] in 1.0s (231.31/s) 


,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,transfer_index,true_sale_index
0,179908482,2024-08-16,RENTAL,LISTING_REMOVED,NaN,0.0,0,1.0,INVITATION_HOMES,1,3,2
1,179908482,2024-08-07,RENTAL,PRICE_CHANGE,1275.0,0.0,0,1.0,INVITATION_HOMES,1,3,2
2,179908482,2024-08-02,RENTAL,PRICE_CHANGE,1295.0,0.0,0,1.0,INVITATION_HOMES,1,3,2
3,179908482,2024-07-23,RENTAL,LISTED_RENT,1350.0,0.0,0,1.0,INVITATION_HOMES,1,3,2
4,179908482,2023-12-08,RENTAL,LISTING_REMOVED,NaN,0.0,0,1.0,INVITATION_HOMES,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...
881,73650368,2014-02-04,RENTAL,PRICE_CHANGE,895.0,1.0,0,1.0,None,0,1,1
882,73650368,2013-12-17,RENTAL,PRICE_CHANGE,975.0,1.0,0,1.0,None,0,1,1
883,73650368,2013-10-21,RENTAL,LISTED_RENT,1075.0,1.0,0,1.0,None,0,1,1
884,69552382,2024-07-03,RENTAL,LISTED_RENT,1150.0,0.0,0,1.0,INVITATION_HOMES,1,2,1


### 4. Prepare the Data for the Current Invitation Homes Properties

In [8]:
# Pulling only the current rent rate (either PRICE_CHANGE or LISTED_RENT) at a given property
filtered_rental_events = rental_events[
    rental_events['event_name'].isin(['PRICE_CHANGE', 'LISTED_RENT'])
]

filtered_rental_events = filtered_rental_events.sort_values(
    by=['parcl_property_id', 'event_date'], ascending=[True, False]
)

latest_events_df = filtered_rental_events.drop_duplicates(
    subset=['parcl_property_id'], keep='first'
)

latest_events_df = latest_events_df[['parcl_property_id', 'price']]

# Display the resulting DataFrame
latest_events_df


,parcl_property_id,price
884,69552382,1150.0
871,73650368,1495.0
870,73996984,1795.0
861,74258825,1350.0
841,75810604,2510.0
...,...,...
82,174969896,1700.0
78,177880533,1700.0
41,178119496,2240.0
19,178350775,1870.0


In [17]:
merged_current_props = current_ih_units_df.merge(latest_events_df, on='parcl_property_id', how='left')

# Calculate inventory and rent rates
current_state_invh = merged_current_props.groupby(['cbsa', 'current_entity_owner_name']).agg(
    Current_Inventory=('parcl_property_id', 'count'),  # Count of 'parcl_property_id'
    Current_Rent_Rates=('price', 'median')  # Median of 'price'
).reset_index()

# Rename columns for the final output format
current_state_invh.rename(columns={
    'cbsa': 'Market',
    'current_entity_owner_name': 'Entity_Name',
}, inplace=True)

#print final current state df
current_state_invh


,Market,Entity_Name,Current_Inventory,Current_Rent_Rates
0,"Cincinnati, OH-KY-IN",INVITATION_HOMES,87,1400.0
1,"Columbia, SC",INVITATION_HOMES,2,1275.0
2,"Columbus, OH",INVITATION_HOMES,124,2015.0
3,"Indianapolis-Carmel-Anderson, IN",INVITATION_HOMES,8,1597.5
4,"St. Louis, MO-IL",INVITATION_HOMES,11,1250.0


### 5. Retrieve the Data for the Nov 2023 Invitation Homes Properties

In [10]:
# Since we want a point in time portfolio analysis (Nov 2023), we cannot pull the current IH portfolio to truncate this query and must grab all units in the five markets, a much larger query
total_market_units = client.property.search.retrieve(
    parcl_ids=market_parcl_id_list,
    property_type='single_family'
)

#Dataframe of properties with CBSA and Current Owner 
total_market_units = total_market_units[['parcl_property_id', 'cbsa', 'cbsa_parcl_id', 'property_type']]

#Parcl Prop ID list to pass into events
total_unit_list = total_market_units['parcl_property_id'].tolist()

print(len(total_unit_list))

Processing Parcl IDs |████████████████████████████████████████| 5/5 [100%] in 3:19.6 (0.03/s) 
2918030


In [11]:
# pass in the parcl_property_id_list to pull in all events in the five markets through November 2023

market_events_2023 = client.property.events.retrieve(
        parcl_property_ids=total_unit_list,
        end_date='2023-11-30'
)

market_events_2023

Processing Parcl Property IDs |████████████████████████████████████████| 2918030/2918030 [100%] in 8:39.6 (5615.44/s) 


,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,transfer_index,true_sale_index
0,278579703,2023-10-10,LISTING,PRICE_CHANGE,239500.0,NaN,0,NaN,None,0,0,0
1,278579703,2023-10-01,LISTING,LISTED_SALE,249500.0,NaN,0,NaN,None,0,0,0
2,278579703,2023-09-22,LISTING,LISTED_SALE,249500.0,NaN,0,NaN,None,0,0,0
3,278579703,2017-03-01,LISTING,LISTING_REMOVED,109000.0,NaN,0,NaN,None,0,0,0
4,278579703,2017-01-20,LISTING,PENDING_SALE,109000.0,NaN,0,NaN,None,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5397050,64835212,2013-03-21,SALE,SOLD,88000.0,0.0,0,1.0,None,0,2,2
5397051,51094428,2022-05-23,SALE,SOLD,270000.0,1.0,0,0.0,None,0,3,2
5397052,51094428,2022-04-08,LISTING,LISTED_SALE,240000.0,1.0,0,0.0,None,0,2,1
5397053,51094428,2019-05-10,SALE,SOLD,169900.0,1.0,0,0.0,None,0,2,1


### 6. Prepare the Data for the November 2023 Invitation Homes Properties

In [12]:
# Pulling only the latest eventat every property prior to November 2023
latest_2023_event = market_events_2023.sort_values(
    by=['parcl_property_id', 'event_date'], ascending=[True, False]
)

latest_2023_event_dedupe = latest_2023_event.drop_duplicates(
    subset=['parcl_property_id'], keep='first'
)

# Display the resulting DataFrame
latest_2023_event_dedupe

,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,transfer_index,true_sale_index
4804415,48767618,2022-07-06,SALE,SOLD,387000.0,1.0,0,0.0,None,1,5,5
5386165,48781893,2022-02-24,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,0.0,None,1,7,5
4887321,48793708,2022-06-23,RENTAL,LISTING_REMOVED,NaN,0.0,0,1.0,AMHERST,0,6,4
5330859,48824819,2022-01-05,SALE,SOLD,139900.0,1.0,0,0.0,None,1,11,10
4452821,48835503,2021-12-23,SALE,SOLD,308000.0,0.0,0,0.0,None,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4649663,472880620,2010-07-02,LISTING,LISTING_REMOVED,219500.0,NaN,0,NaN,None,0,0,0
2714260,472880940,2017-07-14,SALE,SOLD,540000.0,1.0,0,0.0,None,1,1,1
5119538,472881394,2012-02-01,RENTAL,PRICE_CHANGE,1300.0,NaN,0,NaN,None,0,0,0
2700458,472890397,2022-08-15,SALE,SOLD,157500.0,1.0,1,0.0,None,1,1,1


In [13]:
# find the props owned by IH at the time

props_owned_by_ih_nov_2023 = latest_2023_event_dedupe[
    latest_2023_event_dedupe['entity_owner_name'].isin(['INVITATION_HOMES'])
]

props_owned_by_ih_nov_2023

,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,transfer_index,true_sale_index
4676033,65190176,2018-05-14,SALE,SOLD,11302.0,0.0,0,1.0,INVITATION_HOMES,1,1,1
1924045,66985885,2012-12-31,SALE,SOLD,185000.0,0.0,0,1.0,INVITATION_HOMES,0,1,1
4853081,68520508,2022-01-21,SALE,SOLD,250000.0,0.0,0,1.0,INVITATION_HOMES,1,2,1
5171009,69325326,2022-07-06,SALE,SOLD,2705600.0,0.0,0,1.0,INVITATION_HOMES,1,3,3
2700372,69898049,2023-07-19,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3084230,181105053,2014-07-08,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,1,3,2
4503125,181871421,2022-08-31,SALE,SOLD,230000.0,0.0,0,1.0,INVITATION_HOMES,1,3,2
3173244,182833375,2014-10-07,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,1,4,1
4551748,456492864,2017-01-31,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,0,1,0


In [21]:
# Merge the two DataFrames on 'parcl_property_id'
parcls_events_ih = props_owned_by_ih_nov_2023.merge(total_market_units, on='parcl_property_id', how='inner')

# Select only the columns 'parcl_property_id', 'cbsa', 'price', and 'entity_owner_name'
final_events = parcls_events_ih[['parcl_property_id', 'cbsa', 'entity_owner_name', 'transfer_index']]

# Filter latest_2023_event_dedupe for rows where event_type is 'RENTAL' and event_name is either 'LISTED_RENT' or 'PRICE_CHANGE'
rentals_2023 = latest_2023_event_dedupe[
    (latest_2023_event_dedupe['event_type'] == 'RENTAL') &
    (latest_2023_event_dedupe['event_name'].isin(['LISTED_RENT', 'PRICE_CHANGE'])) &
    (latest_2023_event_dedupe['price'] <= 8000)
]

# Select only the columns needed for the join
rentals_2023 = rentals_2023[['parcl_property_id', 'transfer_index', 'price']]

# Perform a left join on final_events with filtered_latest_2023 using 'parcl_property_id' and 'transfer_index'
final_events_with_prices = final_events.merge(
    rentals_2023,
    on=['parcl_property_id', 'transfer_index'],
    how='left',
    suffixes=('', '_rental')  # to avoid column name conflicts if needed
)

final_events_with_prices

,parcl_property_id,cbsa,entity_owner_name,transfer_index,price
0,65190176,"Columbus, OH",INVITATION_HOMES,1,NaN
1,66985885,"Indianapolis-Carmel-Anderson, IN",INVITATION_HOMES,1,NaN
2,68520508,"Columbus, OH",INVITATION_HOMES,2,NaN
3,69325326,"Columbus, OH",INVITATION_HOMES,3,NaN
4,69898049,"Indianapolis-Carmel-Anderson, IN",INVITATION_HOMES,2,NaN
...,...,...,...,...,...
279,181105053,"Cincinnati, OH-KY-IN",INVITATION_HOMES,3,NaN
280,181871421,"Columbus, OH",INVITATION_HOMES,3,NaN
281,182833375,"Cincinnati, OH-KY-IN",INVITATION_HOMES,4,NaN
282,456492864,"Columbus, OH",INVITATION_HOMES,1,NaN


In [22]:
# Calculate IH 2023 inventory and rent rates
invh_2023 = final_events_with_prices.groupby(['cbsa', 'entity_owner_name']).agg(
    November_2023_Inventory=('parcl_property_id', 'count'),  # Count of 'parcl_property_id'
    November_2023_Rent_Rates=('price', 'median')  # Median of 'price'
).reset_index()

# Rename columns for the final output format
invh_2023.rename(columns={
    'cbsa': 'Market',
    'entity_owner_name': 'Entity_Name',
}, inplace=True)

invh_2023

,Market,Entity_Name,November_2023_Inventory,November_2023_Rent_Rates
0,"Cincinnati, OH-KY-IN",INVITATION_HOMES,93,1400.0
1,"Columbia, SC",INVITATION_HOMES,4,NaN
2,"Columbus, OH",INVITATION_HOMES,132,1917.5
3,"Indianapolis-Carmel-Anderson, IN",INVITATION_HOMES,33,1200.0
4,"St. Louis, MO-IL",INVITATION_HOMES,22,1250.0


### 7. Calculate the YoY Change for the Invitation Homes Portfolio in the Tertiary Markets

In [23]:
# Calc the YoY changes
YoY_changes = pd.merge(
    invh_2023,
    current_state_invh,
    on=['Market', 'Entity_Name'],
    how='left'
)

# Calculate YoY Inventory Change and YoY Rent Change
YoY_changes['YoY Inventory Change'] = ((YoY_changes['Current_Inventory'] - YoY_changes['November_2023_Inventory']) 
                                     / YoY_changes['November_2023_Inventory']) * 100
YoY_changes['YoY Rent Change'] = ((YoY_changes['Current_Rent_Rates'] - YoY_changes['November_2023_Rent_Rates']) 
                                / YoY_changes['November_2023_Rent_Rates']) * 100

# Rename columns for the final output
YoY_changes.rename(columns={
    'November_2023_Inventory': 'November 2023 Inventory',
    'Current_Inventory': 'Current Inventory',
    'November_2023_Rent_Rates': 'November 2023 Rent Rates',
    'Current_Rent_Rates': 'Current Rent Rates'
}, inplace=True)

# Reorder columns to match the desired format
final = YoY_changes[['Market', 'Entity_Name', 'November 2023 Inventory', 'Current Inventory', 
                      'YoY Inventory Change', 'November 2023 Rent Rates', 'Current Rent Rates', 
                      'YoY Rent Change']]

# Export the final DataFrame to CSV
final.to_csv('final_market_data.csv', index=False)

# Display the final DataFrame to confirm
final

,Market,Entity_Name,November 2023 Inventory,Current Inventory,YoY Inventory Change,November 2023 Rent Rates,Current Rent Rates,YoY Rent Change
0,"Cincinnati, OH-KY-IN",INVITATION_HOMES,93,87,-6.451613,1400.0,1400.0,0.000000
1,"Columbia, SC",INVITATION_HOMES,4,2,-50.000000,NaN,1275.0,NaN
2,"Columbus, OH",INVITATION_HOMES,132,124,-6.060606,1917.5,2015.0,5.084746
3,"Indianapolis-Carmel-Anderson, IN",INVITATION_HOMES,33,8,-75.757576,1200.0,1597.5,33.125000
4,"St. Louis, MO-IL",INVITATION_HOMES,22,11,-50.000000,1250.0,1250.0,0.000000
